In [8]:
# Imports and paths
from pathlib import Path
from ultralytics import YOLO
import cv2
import numpy as np

ROOT = Path(r"C:/Probook/E/AI/Projects/Image Processing/Vehicels/data/Vehicle_Detection_Image_Dataset")
IMAGES_DIR_VALID = ROOT / "valid" / "images"
IMAGES_DIR_TRAIN = ROOT / "train" / "images"


IMAGES_DIR = IMAGES_DIR_VALID if IMAGES_DIR_VALID.exists() else IMAGES_DIR_TRAIN


In [9]:
# Pick a sample image and video (fallbacks included)

IMG = ROOT / "sample_image.jpg"
if not IMG.exists():
    jpgs = list(IMAGES_DIR.glob("*.jpg"))
    IMG = jpgs[0] if jpgs else None


VID = ROOT / "sample_video.mp4"
if not VID.exists():
    vids = list(ROOT.glob("*.mp4")) + list(ROOT.glob("*.avi"))
    VID = vids[0] if vids else None

print("[Info] Selected image path:", IMG)
print("[Info] Images directory:", IMAGES_DIR)
print("[Info] Selected video path:", VID)

assert IMG is not None and IMG.exists(), "No image found. Double-check ROOT and your files."


[Info] Selected image path: C:\Probook\E\AI\Projects\Image Processing\Vehicels\data\Vehicle_Detection_Image_Dataset\sample_image.jpg
[Info] Images directory: C:\Probook\E\AI\Projects\Image Processing\Vehicels\data\Vehicle_Detection_Image_Dataset\valid\images
[Info] Selected video path: C:\Probook\E\AI\Projects\Image Processing\Vehicels\data\Vehicle_Detection_Image_Dataset\sample_video.mp4


In [10]:
# Model configuration and load

WEIGHTS = "yolov8l.pt"
IMG_SIZE = 1280
CONF = 0.45
IOU  = 0.80
MAX_DET = 1000
DEVICE = 0          
AUGMENT = True
CLASSES = [2, 3, 5, 7]  

print("[Info] Loading model:", WEIGHTS)
model = YOLO(WEIGHTS)
print("[OK] Model loaded.")


[Info] Loading model: yolov8l.pt
[OK] Model loaded.


In [11]:
# Run prediction on a single image

out_dir_img = Path("runs/detect/step1_image_overlap")
_ = model.predict(
    source=str(IMG),
    imgsz=IMG_SIZE,
    conf=CONF,
    iou=IOU,
    classes=CLASSES,
    device=DEVICE,
    max_det=MAX_DET,
    augment=AUGMENT,
    save=True,
    project="runs/detect",
    name="step1_image_overlap",
    exist_ok=True,
    show_conf=False
)
print("[Done] Image results saved to:", out_dir_img.resolve())



image 1/1 C:\Probook\E\AI\Projects\Image Processing\Vehicels\data\Vehicle_Detection_Image_Dataset\sample_image.jpg: 736x1280 6 cars, 3 trucks, 335.1ms
Speed: 6.9ms preprocess, 335.1ms inference, 8.4ms postprocess per image at shape (1, 3, 736, 1280)
Results saved to C:\Users\Asus\ENV\YOLOV8\vehicle detection\runs\detect\step1_image_overlap
[Done] Image results saved to: C:\Users\Asus\ENV\YOLOV8\vehicle detection\runs\detect\step1_image_overlap


In [12]:
# Run prediction on a video 
if VID is None or not Path(VID).exists():
    print("[Skip] No video found. Skipping video step.")
else:
    out_dir_video = Path("runs/detect/step3_video")
    _ = model.predict(
        source=str(VID),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        classes=CLASSES,
        device=DEVICE,
        max_det=MAX_DET,
        augment=False,   
        vid_stride=2,   
        save=True,
        project="runs/detect",
        name="step3_video",
        exist_ok=True,
        show_conf=False,
        verbose=False,
        stream=True
    )
    print("[Done] Video results saved to:", out_dir_video.resolve())


[Done] Video results saved to: C:\Users\Asus\ENV\YOLOV8\vehicle detection\runs\detect\step3_video
